In [17]:
%env CYPHER=/usr/bin/cypher-shell
%env HOST=your_host
%env USER=neo4j
%env PW=your_pw

env: CYPHER=/usr/bin/cypher-shell
env: HOST=your_host
env: USER=neo4j
env: PW=your_pw


In [2]:
CONNECT="$CYPHER -a $HOST -u $USER -p $PW"

In [7]:
from IPython.core.magic import register_cell_magic
import shlex

@register_cell_magic('neo')
def neo(line, cell):
    !{CONNECT} {shlex.quote(cell)}

In [8]:
%%neo
show users

+------------------------------------------------------------------+
| user    | roles      | passwordChangeRequired | suspended | home |
+------------------------------------------------------------------+
| "neo4j" | ["PUBLIC"] | FALSE                  | FALSE     | NULL |
+------------------------------------------------------------------+

1 row
ready to start consuming query after 8 ms, results consumed after another 307 ms


In [9]:
%%neo
MATCH (n) RETURN n LIMIT 5

+--------------------------------------------------------------------------------------+
| n                                                                                    |
+--------------------------------------------------------------------------------------+
| (:Movie {tagline: "Welcome to the Real World", title: "The Matrix", released: 1999}) |
| (:Person {name: "Keanu Reeves", born: 1964})                                         |
| (:Person {name: "Carrie-Anne Moss", born: 1967})                                     |
| (:Person {name: "Laurence Fishburne", born: 1961})                                   |
| (:Person {name: "Hugo Weaving", born: 1960})                                         |
+--------------------------------------------------------------------------------------+

5 rows
ready to start consuming query after 40 ms, results consumed after another 3 ms


In [10]:
%%neo 
MATCH (p:Person)-[:ACTED_IN]->(m:Movie) 
WHERE m.title = 'Top Gun' 
RETURN p

+-------------------------------------------------+
| p                                               |
+-------------------------------------------------+
| (:Person {name: "Val Kilmer", born: 1959})      |
| (:Person {name: "Meg Ryan", born: 1961})        |
| (:Person {name: "Tom Skerritt", born: 1933})    |
| (:Person {name: "Kelly McGillis", born: 1957})  |
| (:Person {name: "Tom Cruise", born: 1962})      |
| (:Person {name: "Anthony Edwards", born: 1962}) |
+-------------------------------------------------+

6 rows
ready to start consuming query after 80 ms, results consumed after another 3 ms


##### Prompt 1: Which persons reviewed a movie? Return the persons' names and the movies which they reviewed. Order by person's name. 

##### Hint: the relationship type is called `REVIEWED`

In [12]:
%%neo
MATCH (p:Person)-[r:REVIEWED]->(m:Movie) 
RETURN p.name, m.title
ORDER BY p.name

+------------------------------------------+
| p.name             | m.title             |
+------------------------------------------+
| "Angela Scope"     | "The Replacements"  |
| "James Thompson"   | "The Replacements"  |
| "James Thompson"   | "The Da Vinci Code" |
| "Jessica Thompson" | "Cloud Atlas"       |
| "Jessica Thompson" | "The Replacements"  |
| "Jessica Thompson" | "Unforgiven"        |
| "Jessica Thompson" | "The Birdcage"      |
| "Jessica Thompson" | "The Da Vinci Code" |
| "Jessica Thompson" | "Jerry Maguire"     |
+------------------------------------------+

9 rows
ready to start consuming query after 85 ms, results consumed after another 2 ms


##### Prompt 2: Which persons reviewed a movie and also follow the same movie's actors? Return the person's name and movie title.

##### Hint: the relationship type is called `FOLLOWS`

In [13]:
%%neo
MATCH (r:Person)-[:REVIEWED]->(m:Movie), 
(a:Person)-[:ACTED_IN]->(m:Movie), 
(r:Person)-[:FOLLOWS]->(a:Person)
RETURN r.name, a.name, m.title;

+---------------------------+
| r.name | a.name | m.title |
+---------------------------+
+---------------------------+

0 rows
ready to start consuming query after 128 ms, results consumed after another 2 ms


##### Prompt 3: Create a `FOLLOWS` relationship from Jessica Thompson to Bonnie Hunt

In [15]:
%%neo
MATCH (r:Person), (a:Person)
WHERE r.name = "Jessica Thompson" AND a.name = "Bonnie Hunt"
CREATE (r)-[f:FOLLOWS]->(a)
RETURN r, f, a;

+-------------------------------------------------------------------------------------------------+
| r                                    | f          | a                                           |
+-------------------------------------------------------------------------------------------------+
| (:Person {name: "Jessica Thompson"}) | [:FOLLOWS] | (:Person {name: "Bonnie Hunt", born: 1961}) |
+-------------------------------------------------------------------------------------------------+

1 row
ready to start consuming query after 144 ms, results consumed after another 61 ms
Created 1 relationships


##### Prompt 4: Re-test the previous query you wrote for Prompt 2

In [16]:
%%neo
MATCH (r:Person)-[:REVIEWED]->(m:Movie), 
(a:Person)-[:ACTED_IN]->(m:Movie), 
(r:Person)-[:FOLLOWS]->(a:Person)
RETURN r.name, a.name, m.title;

+------------------------------------------------------+
| r.name             | a.name        | m.title         |
+------------------------------------------------------+
| "Jessica Thompson" | "Bonnie Hunt" | "Jerry Maguire" |
+------------------------------------------------------+

1 row
ready to start consuming query after 7 ms, results consumed after another 2 ms
